# Structural Data
As the structural values of the lsoa area, describe the background featuers of each lsoa.
- population density
- age groups 
- ...

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

## Population Density

In [ ]:
# reading population density
density19_path = "data/stru_data/SAPE22DT11_mid_2019_lsoa_population_density.xlsx"
pop_den19 = pd.ExcelFile(density19_path)

In [ ]:
# reading sheet
df_den_19 = pop_den19.parse(sheet_name="Mid-2019 Population Density", header = 4) 
df_den_19.head()

,LSOA Code,LSOA Name,Mid-2019 population,Area Sq Km,People per Sq Km
0,E01011949,Hartlepool 009A,1954,0.5189,3765.658123
1,E01011950,Hartlepool 008A,1257,0.1325,9486.792453
2,E01011951,Hartlepool 007A,1209,0.2086,5795.781400
3,E01011952,Hartlepool 002A,1740,0.4641,3749.191984
4,E01011953,Hartlepool 002B,2033,0.8833,2301.596287


In [ ]:
# select columns
den19_col = ['LSOA Code', 'Mid-2019 population', 'People per Sq Km']
den_19 = df_den_19[den19_col]

In [7]:
# reading the density for 2015
density15_path = "data/stru_data/SAPE20DT11-mid-2015-lsoa-population-density.xls"
pop_den15 = pd.ExcelFile(density15_path)
df_den_15 = pop_den15.parse(sheet_name="Mid-2015 Population Density", header = 4) 
df_den_15.head()

,Code,Name,Mid-2015 population,Area Sq Km,People per Sq Km
0,E01020634,County Durham 001A,1524,6.4975,235
1,E01020635,County Durham 001B,1315,1.6963,775
2,E01020636,County Durham 001C,1749,0.9272,1886
3,E01020654,County Durham 001D,1860,0.9884,1882
4,E01020676,County Durham 001E,1479,5.4717,270


In [9]:
# select columns
den15_col = ['Code', 'Mid-2015 population', 'People per Sq Km']
den_15 = df_den_15[den15_col]

In [13]:
# join together
# 把 df2 的列名改成一致
den_15.rename(columns={'Code': 'LSOA Code'}, inplace=True)
den_15.rename(columns={'People per Sq Km': '2015_population_density'}, inplace=True)
den_19.rename(columns={'People per Sq Km': '2019_population_density'}, inplace=True)

# 然后 merge
den_all = den_19.merge(den_15, on='LSOA Code', how='left')
print(den_all.head())


   LSOA Code  Mid-2019 population  2019_population_density  \
0  E01011949                 1954              3765.658123   
1  E01011950                 1257              9486.792453   
2  E01011951                 1209              5795.781400   
3  E01011952                 1740              3749.191984   
4  E01011953                 2033              2301.596287   

   Mid-2015 population  2015_population_density  
0                 2009                     3872  
1                 1443                    10891  
2                 1279                     6131  
3                 1745                     3760  
4                 2011                     2277  


C:\Users\hp\AppData\Local\Temp\ipykernel_27232\3006015090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  den_15.rename(columns={'Code': 'LSOA Code'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_27232\3006015090.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  den_15.rename(columns={'People per Sq Km': '2015_population_density'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_27232\3006015090.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [14]:
den_all['pop_growth_rate'] = (den_all['Mid-2019 population'] - den_all['Mid-2015 population']) / den_all['Mid-2015 population']
den_all['avg_den'] = (den_all['2015_population_density'] + den_all['2019_population_density']) / 2
print(den_all.head())

   LSOA Code  Mid-2019 population  2019_population_density  \
0  E01011949                 1954              3765.658123   
1  E01011950                 1257              9486.792453   
2  E01011951                 1209              5795.781400   
3  E01011952                 1740              3749.191984   
4  E01011953                 2033              2301.596287   

   Mid-2015 population  2015_population_density  pop_growth_rate       avg_den  
0                 2009                     3872        -0.027377   3818.829061  
1                 1443                    10891        -0.128898  10188.896226  
2                 1279                     6131        -0.054730   5963.390700  
3                 1745                     3760        -0.002865   3754.595992  
4                 2011                     2277         0.010940   2289.298143  


In [15]:
den_all.isna().sum()

LSOA Code                  0
Mid-2019 population        0
2019_population_density    0
Mid-2015 population        0
2015_population_density    0
pop_growth_rate            0
avg_den                    0
dtype: int64

In [ ]:
# read in lsoa shapefile
london_gdf = gpd.read_file("data/statistical-gis-boundaries-london/statistical-gis-boundaries-london/ESRI/LSOA_2011_London_gen_MHW.shp")

# extract lsoa11cd as LSOA list
london_lsoa_list = london_gdf['LSOA11CD'].unique()

# select data in London area
df_london_den = den_all[den_all['LSOA Code'].isin(london_lsoa_list)]
df_london_den.shape

(4835, 7)

: 